### Grid read & to point
#### CDO를 이용하여 /DAIO/nwp/stn/{stn_id}" 경로에 지점 추출자료 생성(nc포맷)

In [4]:
import numpy as np
import pandas as pd 

# set para
#stn_latlon = {"47105": [37.7515, 128.891]}
stn_latlon = {'875': [38.1211, 128.4606]}

print(stn_latlon.items())
print(len(stn_latlon))

dict_items([('875', [38.1211, 128.4606])])
1


#### grid to point

In [7]:
#!pip install cdo
from cdo import Cdo
#import cdo
import glob
import sys
import os

#
cdo = Cdo()


# Get all files in the current directory
#input_dir = "/Users/mankikim/JOB/DATA/"    # for mac
#input_dir = "/Users/mankikim/Desktop/"
input_dir = "/Users/mankikim/Desktop/"
#input_dir = "D:/KMK_DATA/NWP/"    # for tjpc

files = glob.glob(input_dir + "*.nc")

# nearest grid 추출
for i, file in enumerate(files):
    date = file.split(".")[1]
    for key, value in stn_latlon.items():
        print(i, key, value, file)
        stn_id = key
        lat = value[0]
        lon = value[1]

        cdo_parameters = f'lon={lon}_lat={lat}'

        if not os.path.exists(f"../DAIO/nwp/stn/{key}"): os.makedirs(f"../DAIO/nwp/stn/{key}")

        output_name = f"../DAIO/nwp/stn/{key}/ldaps_stn_{key}_{date}.nc"

        # 파일의 존재 여부를 확인합니다.
        if os.path.exists(output_name):
            print(f"파일이 존재합니다. continue {output_name}")
            continue
        else:
            #print("파일이 존재하지 않습니다.")
            cdo.remapnn(cdo_parameters, input=file, output=output_name)

0 875 [38.1211, 128.4606] /Users/mankikim/Desktop/l015_v070_erlo_unis_20kind.2023010500.nc
Error in calling operator remapnn with:
>>> cdo -O -s -remapnn,lon=128.4606_lat=38.1211 /Users/mankikim/Desktop/l015_v070_erlo_unis_20kind.2023010500.nc ../DAIO/nwp/stn/875/ldaps_stn_875_2023010500.nc<<<
STDOUT:
STDERR:cdo    remapnn: Open failed on >/Users/mankikim/Desktop/l015_v070_erlo_unis_20kind.2023010500.nc<
                Unsupported file type



CDOException: (returncode:1) cdo    remapnn: Open failed on >/Users/mankikim/Desktop/l015_v070_erlo_unis_20kind.2023010500.nc<
                Unsupported file type


#### Extract Lat, Lon

In [32]:
import numpy as np
import netCDF4 as nc

# Get all files in the current directory
input_dir = "/Users/mankikim/JOB/DATA/"
files = glob.glob(input_dir + "*.nc")
files[0]

# nearest grid 추출
latlon_file = nc.Dataset(files[0])
nc_lat = np.array(latlon_file.variables['latitude'])
nc_lon = np.array(latlon_file.variables['longitude'])
with open('../DAIO/nwp/ldaps_lat_grid', 'wb') as f:
    np.save(f, nc_lat)
with open('../DAIO/nwp/ldaps_lon_grid', 'wb') as f:
    np.save(f, nc_lon)

#### 변수리스트 확인

In [3]:
import netCDF4 as nc
test_file = "../DAIO/nwp/stn/47105/ldaps_stn_47105_2021011400.nc"
test_data = nc.Dataset(test_file)

#test_data.variables

for i,nam in enumerate(test_data.variables.keys()):
    print(i, nam)
print(np.array(test_data.variables['time']))
test_data.variables['time']

0 time
1 lon
2 lat
3 NDNSW_surface
4 NCPCP_surface
5 SNOL_surface
6 UGRD_10maboveground
7 VGRD_10maboveground
8 TMP_1_5maboveground
9 TMIN_1_5maboveground
10 TMAX_1_5maboveground
11 SPFH_1_5maboveground
12 RH_1_5maboveground
13 VIS_1_5maboveground
14 DPT_1_5maboveground
15 MAXGUST_0maboveground
16 LCDC_entireatmosphere_consideredasasinglelayer_
17 MCDC_entireatmosphere_consideredasasinglelayer_
18 HCDC_entireatmosphere_consideredasasinglelayer_
19 TCAR_entireatmosphere_consideredasasinglelayer_
20 PRMSL_meansealevel
21 TMP_surface
22 PRES_surface
[1.6105824e+09 1.6105860e+09 1.6105896e+09 1.6105932e+09 1.6105968e+09
 1.6106004e+09 1.6106040e+09 1.6106076e+09 1.6106112e+09 1.6106148e+09
 1.6106184e+09 1.6106220e+09 1.6106256e+09 1.6106292e+09 1.6106328e+09
 1.6106364e+09 1.6106400e+09 1.6106436e+09 1.6106472e+09 1.6106508e+09
 1.6106544e+09 1.6106580e+09 1.6106616e+09 1.6106652e+09 1.6106688e+09
 1.6106724e+09 1.6106760e+09 1.6106796e+09 1.6106832e+09 1.6106868e+09
 1.6106904e+09 1.6106

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: verification time generated by wgrib2 function verftime()
    units: seconds since 1970-01-01 00:00:00.0 0:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (49,)
filling off

#### Make [batch, sequence, features] using point nc

In [14]:
import numpy as np
import netCDF4 as nc
import glob

# .. 날짜 초기화
set_dates = pd.date_range('2021-01-14 09:00:00', '2023-05-31 09:00:00', freq='D')
set_dates = set_dates.strftime("%Y%m%d00")
#print("data day수: ", len(data_dates))


# 추출 변수 셋, 변수 설정 다하기 
col_list= ['NDNSW_surface',
'NCPCP_surface',
'SNOL_surface',
'UGRD_10maboveground',
'VGRD_10maboveground',
'TMP_1_5maboveground',
'TMIN_1_5maboveground',
'TMAX_1_5maboveground',
'SPFH_1_5maboveground',
'RH_1_5maboveground',
'VIS_1_5maboveground',
'DPT_1_5maboveground',
'MAXGUST_0maboveground',
'LCDC_entireatmosphere_consideredasasinglelayer_',
'MCDC_entireatmosphere_consideredasasinglelayer_',
'HCDC_entireatmosphere_consideredasasinglelayer_',
'TCAR_entireatmosphere_consideredasasinglelayer_',
'PRMSL_meansealevel',
'TMP_surface',
'PRES_surface']

# .. nwp 변수 초기화
nwp_data = np.full( (len(set_dates),49,len(col_list)), np.nan )

# 있는 파일만 읽어 인덱스로 입력하기, nan으로 배열이 초기화 되어 있으니까 날짜에 대한 파일을 전부다 조회할 필요 x
for key, value in stn_latlon.items(): 

   # 지점 추출 nwp file
   input_dir = f"../DAIO/nwp/stn/{key}/"
   files = glob.glob(input_dir + "*.nc")
   #print(files)

   for i,file in enumerate(files):
         
         date = file.split("_")[3].split(".")[0]
         #print("date", date)
         index = set_dates.get_loc(date)
         
         # file set
         try:
            ncfile = nc.Dataset(file)
            #ncfile
            print("Read: ", file)

            #if i == 0:
            #    nc_lat = ncfile.variables['lat']
            #    nc_lon = ncfile.variables['lon']

            for j, var in enumerate(col_list):
               a = np.array(ncfile.variables[var]).flatten()
               #nwp_data[index,:,j] = a[:-1] # 0 ~ +47
               nwp_data[index,:,j] = a[:] # 0 ~ +48

         except Exception as e:
            print(e)

   # save
   with open(f'../DAIO/nwp_data_{key}', 'wb') as f:
      np.save(f, nwp_data)

있어야할 day수:  868
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011400.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011500.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011600.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011700.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011800.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021011900.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012000.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012100.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012200.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012300.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012400.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012500.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012600.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012700.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012800.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_47105_2021012900.nc
Read:  ../DAIO/nwp/stn/47105\ldaps_stn_4

In [10]:
ncfile.variables['lon']

<class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (1,)
filling off

In [28]:
print("NWP shape: ", nwp_data.shape)
print("설정 기간에 대한 일수: ", len(set_dates))
index = set(np.where(np.isnan(nwp_data))[0])
print(sorted(index))
print("NWP 결측일수: ", len(index))
print("설정일 - NWP 결측일=", len(set_dates) -len(index))
print("NWP 파일개수: ", len(files))

NWP shape:  (868, 49, 20)
설정 기간에 대한 일수:  868
[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 82, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,

In [84]:
# .. nco read
#from nco import Nco
#import nco
#a = nco.Nco()
#nco = Nco()
#temperatures = nco.ncra(input=file, returnArray=True).variables['NCPCP_surface'][:]




dict_keys(['time', 'lon', 'lat', 'NDNSW_surface', 'NCPCP_surface', 'SNOL_surface', 'UGRD_10maboveground', 'VGRD_10maboveground', 'TMP_1_5maboveground', 'TMIN_1_5maboveground', 'TMAX_1_5maboveground', 'SPFH_1_5maboveground', 'RH_1_5maboveground', 'VIS_1_5maboveground', 'DPT_1_5maboveground', 'MAXGUST_0maboveground', 'LCDC_entireatmosphere_consideredasasinglelayer_', 'MCDC_entireatmosphere_consideredasasinglelayer_', 'HCDC_entireatmosphere_consideredasasinglelayer_', 'TCAR_entireatmosphere_consideredasasinglelayer_', 'PRMSL_meansealevel', 'TMP_surface', 'PRES_surface'])
(48,)
